# Models tuning

In this notebook, we will perform hyperparameter tuning and feature selection to optimize the performance of various machine learning models for power generation prediction. 
We will use techniques like cross-validation and different feature subsets to refine our models. Our goal is to identify the most effective feature combinations and hyperparameter settings to improve model accuracy.


## Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from functools import partial
import warnings
import os
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

## Load data

### Load the historical generation and weather training data that was augmented using feature engineering in the previous notebook

In [ ]:
PRE_PROCESSED_DATA_PATH = os.path.join('..','..', 'data', '1_pre_processed_data.parquet')
GEN_DATA_PATH = os.path.join('..','..', 'data', '2_feature_engineered_data.parquet')
df = pd.read_parquet(GEN_DATA_PATH)
df_without_feature_engineering = pd.read_parquet(PRE_PROCESSED_DATA_PATH)
TARGET_COL = 'DC Gen. Power'
display(df.head(5))

In [ ]:
x = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(x)
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

print('All: ', len(df))

# Training

This section focuses on training machine learning models for power generation prediction. We will explore various algorithms, optimize their hyperparameters, and evaluate their performance to identify the best model for this task.

In [ ]:
results = pd.DataFrame()
def calculate_metrics(y_pred, y_val, model_name, hyperparams, fold, features):
    """
    Calculate metrics for the model
    """
    global results
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)    

    df = pd.DataFrame({
        'model': [model_name],        
        'mae': [mae],
        'mse': [mse],
        'mae': [mae],
        'hyperparams': [hyperparams],
        'fold': [fold],
        'features': [features]
    })
    results = pd.concat([results, df])   

    return mse, rmse, mae

def plot_predictions(y_pred, y_actual, n=200):
    plt.figure(figsize=(20, 8))
    plt.plot(y_actual[-n:], label='Actual')
    plt.plot(y_pred[-n:], label='Predicted')
    plt.legend()
    plt.show()


## Feature Selection

Feature selection is a crucial step in the modeling process that involves choosing the most relevant features from the dataset to improve model performance and reduce complexity. By selecting a subset of features, we aim to enhance the model's ability to generalize, reduce overfitting, and expedite the training process.

In this notebook, we experiment with various subsets of features to evaluate their impact on model performance. We will explore different feature sets, such as all available features, basic features, important features identified through domain knowledge, and specific subsets excluding certain types of features like lags or rolling metrics. This approach allows us to identify which features contribute most significantly to the predictive power of the model and optimize feature selection for better results.

We will employ K-Fold Cross Validation to assess the effectiveness of these feature subsets and determine how they influence model accuracy and robustness across different machine learning algorithms.


In [ ]:

essential_features = [
    'Temperature', 'Shortwave Radiation', 'Longwave Radiation', 'UV Radiation', 
    'Direct Shortwave Radiation', 'Diffuse Shortwave Radiation'
]

all_features = x.columns.tolist()

basic_features = df_without_feature_engineering.columns.tolist()

temporal_features = [
    'day', 'season_0', 'season_1', 'season_2', 'season_3', 
    'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 
    'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 
    'month_11', 'month_12', 'hour_0', 'hour_1', 'hour_2', 
    'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 
    'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 
    'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 
    'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 
    'hour_23', 'Hours Since Last Rain'
]

lag_features = [
    'DC Gen. Power 1 Hour Lag', 'DC Gen. Power 2 Hour Lag', 'DC Gen. Power 4 Hour Lag', 
    'DC Gen. Power 24 Hour Lag', 'DC Gen. Power 720 Hour Lag', 
    'DC Gen. Power 24 Hour Rolling Mean', 'DC Gen. Power 24 Hour Rolling Std', 
    'DC Gen. Power 24 Hour Rolling Max', 'DC Gen. Power 24 Hour Rolling EMA', 
    'DC Gen. Power 48 Hour Rolling Mean', 'DC Gen. Power 48 Hour Rolling Std', 
    'DC Gen. Power 48 Hour Rolling Max', 'DC Gen. Power 48 Hour Rolling EMA', 
    'DC Gen. Power 720 Hour Rolling Mean', 'DC Gen. Power 720 Hour Rolling Std', 
    'DC Gen. Power 720 Hour Rolling Max', 'DC Gen. Power 720 Hour Rolling EMA'
]

rolling_features = [
    'Shortwave Radiation 24 Hour Rolling Mean', 'Shortwave Radiation 24 Hour Rolling Std', 
    'Shortwave Radiation 24 Hour Rolling Max', 'Shortwave Radiation 24 Hour Rolling EMA', 
    'Wind Speed 24 Hour Rolling Mean', 'Wind Speed 24 Hour Rolling Std', 
    'Wind Speed 24 Hour Rolling Max', 'Wind Speed 24 Hour Rolling EMA', 
    'Temperature 24 Hour Rolling Mean', 'Temperature 24 Hour Rolling Std', 
    'Temperature 24 Hour Rolling Max', 'Temperature 24 Hour Rolling EMA', 
    'Relative Humidity 24 Hour Rolling Mean', 'Relative Humidity 24 Hour Rolling Std', 
    'Relative Humidity 24 Hour Rolling Max', 'Relative Humidity 24 Hour Rolling EMA', 
    'Shortwave Radiation 48 Hour Rolling Mean', 'Shortwave Radiation 48 Hour Rolling Std', 
    'Shortwave Radiation 48 Hour Rolling Max', 'Shortwave Radiation 48 Hour Rolling EMA', 
    'Wind Speed 48 Hour Rolling Mean', 'Wind Speed 48 Hour Rolling Std', 
    'Wind Speed 48 Hour Rolling Max', 'Wind Speed 48 Hour Rolling EMA', 
    'Temperature 48 Hour Rolling Mean', 'Temperature 48 Hour Rolling Std', 
    'Temperature 48 Hour Rolling Max', 'Temperature 48 Hour Rolling EMA', 
    'Relative Humidity 48 Hour Rolling Mean', 'Relative Humidity 48 Hour Rolling Std', 
    'Relative Humidity 48 Hour Rolling Max', 'Relative Humidity 48 Hour Rolling EMA', 
    'Shortwave Radiation 720 Hour Rolling Mean', 'Shortwave Radiation 720 Hour Rolling Std', 
    'Shortwave Radiation 720 Hour Rolling Max', 'Shortwave Radiation 720 Hour Rolling EMA', 
    'Wind Speed 720 Hour Rolling Mean', 'Wind Speed 720 Hour Rolling Std', 
    'Wind Speed 720 Hour Rolling Max', 'Wind Speed 720 Hour Rolling EMA', 
    'Temperature 720 Hour Rolling Mean', 'Temperature 720 Hour Rolling Std', 
    'Temperature 720 Hour Rolling Max', 'Temperature 720 Hour Rolling EMA', 
    'Relative Humidity 720 Hour Rolling Mean', 'Relative Humidity 720 Hour Rolling Std', 
    'Relative Humidity 720 Hour Rolling Max', 'Relative Humidity 720 Hour Rolling EMA'
]

seasonal_temporal_features = [
    'day', 'season_0', 'season_1', 'season_2', 'season_3', 
    'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 
    'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 
    'month_11', 'month_12', 'hour_0', 'hour_1', 'hour_2', 
    'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 
    'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 
    'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 
    'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 
    'hour_23'
]

domain_knowledge_features =  ['Hours Since Last Rain','days_since_installation','Wind Chill','Solar Zenith Angle']

FEATURE_SETS = {
    'all': all_features,
    'basic': essential_features + basic_features,
    'temporal': essential_features + temporal_features,
    'lag': essential_features + lag_features,
    'rolling': essential_features + rolling_features,
    'seasonal_temporal': essential_features + seasonal_temporal_features,
    'domain_knowledge': essential_features + domain_knowledge_features
}


## K-Fold Cross Validation


In this section, we perform K-Fold Cross Validation to evaluate and optimize the performance of various machine learning models. K-Fold Cross Validation is a technique that involves partitioning the dataset into \( k \) distinct folds, where each fold is used once as a validation set while the remaining \( k-1 \) folds are used for training. This process is repeated \( k \) times, ensuring that each data point gets to be in a validation set exactly once. This approach helps in assessing the model's ability to generalize to new, unseen data and in tuning hyperparameters to enhance model performance.

We will use this method to test a range of models, our objective is to find the most effective model and hyperparameter settings by comparing their performance across different feature subsets and folds.


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5, test_size=int(0.12*len(x)))
results = pd.DataFrame()

def run_k_fold_cv(model_factory, space, max_evals=100):
    global best_loss, best_model, y_scaler, results
 
    best_loss = np.inf
    best_model = None
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):
        for features_name, feature_set in FEATURE_SETS.items(): 
            x_fold = x.drop(columns=[col for col in x.columns if col not in feature_set])
            x_fold_scaled = x_scaler.fit_transform(x_fold)
            print(f"Fold {fold + 1}")
            print(f"Train: {len(train_idx)}")
            print(f"Validation: {len(val_idx)}")
            x_train_fold, x_val_fold = x_fold_scaled[train_idx], x_fold_scaled[val_idx]
            y_train_fold, y_val_fold = y_scaled[train_idx], y[val_idx]
            
            def objective(params):
                model = model_factory(**params)
                model.fit(x_train_fold, y_train_fold)
                y_pred = model.predict(x_val_fold)
                y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
                mae = mean_absolute_error(y_val_fold, y_pred)
                model_name = model.__class__.__name__
                calculate_metrics(y_pred, y_val_fold, model_name, params, fold +1, features_name)
                global best_loss, best_model
                if mae < best_loss:
                    best_loss = mae
                    best_model = model
                    
                return {'loss': mae, 'status': STATUS_OK}
            
            trials = Trials()
            fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    
    return best_model




## Linear models

In [ ]:
model_factory = lambda **params: LinearRegression(**params)

space = {
    'fit_intercept': hp.choice('fit_intercept', [True]),
}

run_k_fold_cv(model_factory, space, max_evals=2)
# y_pred = best_model.predict(x_scaled)
# y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))
# plot_predictions(y_pred, y.values, 1000)


### ElasticNet

In [ ]:
model_factory = lambda **params: ElasticNet(**params)
space = {
    'alpha': hp.uniform('alpha', 0.01, 1.0),
    'l1_ratio': hp.uniform('l1_ratio', 0.01, 1.0),
    'fit_intercept': hp.choice('fit_intercept', [True]),
    'tol': hp.uniform('tol', 0.00001, 0.1),
    'selection': hp.choice('selection', ['cyclic']),
    'max_iter': hp.choice('max_iter', [100, 200, 400, 1000 ]),
}

best_model = run_k_fold_cv(model_factory, space, max_evals=10)

## Decision trees


### Random Forest Regressors
Random Forests (RFs) leverage ensemble learning, a technique that combines predictions from multiple decision tree regressors (DTRs) to achieve improved model robustness and potentially superior prediction accuracy.

In [ ]:
model_factory = lambda **params: RandomForestRegressor(**params)

space = {
    'n_estimators': hp.choice('n_estimators', [10, 20, 40]),
    'max_depth': hp.choice('max_depth', [None, 4, 8, 16, 32, 64, 128]),
    'min_samples_split': hp.choice('min_samples_split', [10, 20, 40]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [2, 4, 8]),
    'max_features': hp.choice('max_features', [ 8, 16, 32, 64]),
}

best_model = run_k_fold_cv(model_factory, space, max_evals=5)

### Gradient Boost Machine

In [ ]:
model_factory = lambda **params: xgb.XGBRegressor(**params, n_jobs=-1)
space = {
    'n_estimators': hp.choice('n_estimators', [50]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth': hp.uniformint('max_depth', 3, 64),
    'min_child_weight': hp.uniformint('min_child_weight', 1, 8),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
}

best_model = run_k_fold_cv(model_factory, space, max_evals=10)



### CatBoost Regressor

CatBoost is a powerful gradient boosting library that is optimized for categorical features and robust to various data types. Developed by Yandex, CatBoost excels in handling complex datasets with categorical variables and provides high-performance predictive modeling. 

In [ ]:
from catboost import CatBoostRegressor

model_factory = lambda **params: CatBoostRegressor(**params, verbose=False)
space = {
    'iterations': hp.choice('iterations', [50, 100, 200, 400]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'depth': hp.uniformint('depth', 3, 10),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
    'border_count': hp.uniformint('border_count', 32, 255),
}

best_model = run_k_fold_cv(model_factory, space, max_evals=10)


## Neural Networks

In [ ]:
# MLPRegressor
from sklearn.neural_network import MLPRegressor
model_factory = lambda **params: MLPRegressor(**params)
space = {
    'hidden_layer_sizes': hp.choice('hidden_layer_sizes', [(16,), (16, 16), (16, 16, 16), (16, 16, 16, 16), (32,), (32, 32), (64,), (64, 64)]),
    'activation': hp.choice('activation', ['relu']),
    'solver': hp.choice('solver', ['adam']),
    'alpha': hp.uniform('alpha', 0.0001, 0.1),
    'batch_size': hp.choice('batch_size', [32, 64, 128]),
    'learning_rate': hp.choice('learning_rate', ['constant', 'invscaling', 'adaptive']),
    'learning_rate_init': hp.uniform('learning_rate_init', 0.0001, 0.1),
    'max_iter': hp.choice('max_iter', [10000]),
    'early_stopping': hp.choice('early_stopping', [True]),
    'n_iter_no_change': hp.choice('n_iter_no_change', [1, 5, 10]),
}

best_model = run_k_fold_cv(model_factory, space, max_evals=10)

# Results

In [ ]:
display(results.sort_values('mae').head(20))
results_group_all_folds = results.groupby(['model', 'features']).mae.mean().sort_values()
results_grouped = results_group_all_folds.groupby(['model', 'features']).min().sort_values()

plt.figure(figsize=(20, 8))
results_grouped.plot(kind='bar')

plt.title('Model Performance')
plt.ylabel('MAE')
plt.xlabel('Model')
plt.xticks(rotation=45)

plt.show()
display(results_grouped)

In [57]:
results_group_all_folds

model                  features  
CatBoostRegressor      no_rolling               4,747.36
                       no_lag                   4,958.03
RandomForestRegressor  important                5,092.13
                       no_rolling               5,169.69
                       no_hour                  5,261.82
CatBoostRegressor      all                      5,271.81
XGBRegressor           important                5,329.35
                       all                      5,616.61
                       no_rolling               5,620.90
RandomForestRegressor  all                      5,669.29
CatBoostRegressor      no_hour                  5,669.87
XGBRegressor           no_hour                  5,691.27
RandomForestRegressor  basic                    5,973.93
                       no_lag                   6,028.52
MLPRegressor           no_hour                  6,038.15
                       important                6,053.46
XGBRegressor           no_lag                   6,421.